# Given a Bank customer, can we build a classifier which can determine whether they will leave in the next 6 months or not?

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Let's import all necessary libraries for our project
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score, ShuffleSplit
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_curve, roc_auc_score, precision_score, recall_score
from sklearn.utils import shuffle
!pip install plot_metric
from plot_metric.functions import BinaryClassification

In [ ]:
#Let's open our dataset
df = pd.read_csv('/kaggle/input/bank-customer-churn-modeling/Churn_Modelling.csv')
display(df.head(5))
display(df.tail(5))

In [ ]:
# Let's see what we have there
display(df.info())
display(df.describe())

In [ ]:
#Do we have null entries
df.isnull().sum()

In [ ]:
#What about duplicates?
df.duplicated().sum()

In [ ]:
#Let's see corr table
df.corr()

In [ ]:
#How many males and females do we have
df['Gender'].value_counts()

In [ ]:
#What countries do we have
df['Geography'].value_counts()

In [ ]:
#Let's delete columns that we don't need for our future model
df = df.drop(['RowNumber','CustomerId','Surname'], axis=1)

In [ ]:
df.head()

In [ ]:
#Encode categorical features
df = pd.get_dummies(df, drop_first=True)

#Standardizing features
scaler = StandardScaler()
numeric = ['CreditScore', 'Age', 'Balance', 'EstimatedSalary']
scaler.fit(df[numeric])
df[numeric] = scaler.transform(df[numeric])

In [ ]:
df

In [ ]:
#Splitting our dataset into training, validating and test sets
X = df.drop(['Exited'], axis=1)
y = df['Exited']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.6, test_size=0.4, random_state=12345)
X_test, X_valid, y_test, y_valid = train_test_split(X_valid, y_valid, test_size=0.5, random_state=12345)

print('Training set size:', X_train.shape[0])
print('Validating set size:', X_valid.shape[0])
print('Test set size:',X_test.shape[0])

# RandomForestClassifier model

In [ ]:
model = RandomForestClassifier(random_state=12345)
model.fit(X_train, y_train)
predicted_valid = model.predict(X_valid)
accuracy_valid = accuracy_score(predicted_valid, y_valid)
print('Accuracy is:', accuracy_valid)
print('F1_Score is:', f1_score(y_valid, predicted_valid))
confusion_matrix(y_valid, predicted_valid)

# LogisticRegression model

In [ ]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(X_train, y_train)
predicted_valid = model.predict(X_valid)
accuracy_valid = accuracy_score(predicted_valid, y_valid)
print('Accuracy is:', accuracy_valid)
print('F1_Score is:', f1_score(y_valid, predicted_valid))

confusion_matrix(y_valid, predicted_valid)

#### RandomForestClassifier shows best results, let's use it for our prediction.

In [ ]:
#let's use "loop" for find the best parameters for our RandomForestClassifier to get best accuracy and F1_score
for depth,estimator in itertools.product(range(1,30,10), range(1,151,50)):
    model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=estimator, class_weight='balanced')
    model.fit(X_train, y_train)
    predictions = model.predict(X_valid)
    score = accuracy_score(predictions, y_valid)
    fscore = f1_score(y_valid, predictions)
    print('Depth: {: >5.0f}       |   Estimators: {:>5.0f}'.format(depth, estimator))
    print('Accuracy: {: >5.4f}   |   F1_Score: {: >5.4f}'.format(score,fscore))
    print('------------------------------------------------------')

<b>Well it looks like with max_depth = 11 and n_estimators = 101 we got f1_score 0.5985 and accuracy score 0.8410, let's try to improve our model</b>

In [ ]:
#Let's try to increase our training sets

def upsampling(features, target, repeat):
    X_zeros = X_train[y_train == 0]
    X_ones = X_train[y_train == 1]
    y_zeros = y_train[y_train == 0]
    y_ones = y_train[y_train == 1]

    X_upsampled = pd.concat([X_zeros] + [X_ones] * repeat)
    y_upsampled = pd.concat([y_zeros] + [y_ones] * repeat)
    
    X_upsampled, y_upsampled = shuffle(X_upsampled, y_upsampled, random_state=12345)
    
    return X_upsampled, y_upsampled


for repeat in range(1,10,1):
    X_upsampled, y_upsampled = upsampling(X_train, y_train, repeat)

    model = RandomForestClassifier(random_state=12345)
    model.fit(X_upsampled, y_upsampled)
    predictions = model.predict(X_valid)
    print("F1_Score: {: >5.3f}  |   Repeats : {: >5.0f}".format(f1_score(y_valid, predictions),repeat))

In [ ]:
#Let's try to decrease our training set

def downsampling(X, y, fraction):
    X_zeros = X[y == 0]
    X_ones = X[y == 1]
    y_zeros = y[y == 0]
    y_ones = y[y == 1]

    X_downsampled = pd.concat(
        [X_zeros.sample(frac=fraction, random_state=12345)] + [X_ones])
    y_downsampled = pd.concat(
        [y_zeros.sample(frac=fraction, random_state=12345)] + [y_ones])
    
    X_downsampled, y_downsampled = shuffle(
        X_downsampled, y_downsampled, random_state=12345)
    
    return X_downsampled, y_downsampled

for i in np.arange(0.03, 0.31, 0.03):
    X_downsampled, y_downsampled = downsampling(X_train, y_train, i)
    model = RandomForestClassifier(random_state=12345)
    model.fit(X_downsampled, y_downsampled)
    predictions = model.predict(X_valid)
    print("F1_Score: {: >5.3f}  |  Fractions : {: >5.3f}".format(f1_score(y_valid, predictions), i))

Well, we didn't improve our model with increasing or decreasing training sets. so let's use parameters that we found in previous loop cycle.

# Testing our model

In [ ]:
# Let's use best parameters for our test set
model = RandomForestClassifier(n_estimators = 101, max_depth=11, class_weight='balanced', random_state=12345)
model.fit(X_train, y_train)
predicted_valid = model.predict(X_test)

accuracy_valid = accuracy_score(predicted_valid, y_test)
print('Accuracy is:', accuracy_valid)
print('F1_Score is:', f1_score(y_test, predicted_valid))
print('Precission is: {:.3f}'.format(precision_score(y_test, predicted_valid)))
print('Recall is: {:.3f}'.format(recall_score(y_test, predicted_valid)))

In [ ]:
#For better learning let's merge our train and valid sets.
final_X = pd.concat([X_train]+[X_valid])
final_y = pd.concat([y_train]+[y_valid])

model = RandomForestClassifier(n_estimators = 101, max_depth=11, class_weight='balanced', random_state=12345)
model.fit(final_X, final_y)
predicted_valid = model.predict(X_test)

accuracy_valid = accuracy_score(predicted_valid, y_test)
print('Accuracy is: {:.3f}'.format(accuracy_valid))
print('F1_Score is: {:.3f}'.format(f1_score(y_test, predicted_valid)))
print('Precission is: {:.3f}'.format(precision_score(y_test, predicted_valid)))
print('Recall is: {:.3f}'.format(recall_score(y_test, predicted_valid)))

In [ ]:
#ROC AUC
y_pred = model.predict_proba(X_test)[:,1]
bc = BinaryClassification(y_test, y_pred, labels=["Class 1"])

plt.figure(figsize=(8,8))
bc.plot_roc_curve()
plt.show()
print('                       Roc Auc Score is: {:.3f}'.format(roc_auc_score(y_test, y_pred)))

### Conclusion:
- We tried two models: RandomForestClassifier and LogisticRegression, at the initial stage LogisticRegression showed the worst results, so we chose RandomForestClassifier for further work.
- With best parameters that we found we got f1_score: 0.625 and Accuracy score: 0.84. Well it looks like our model predicts well.
